In [ ]:
from sr.volvox import analyze, load_data
from sr.volvox.io import get_xyz_data
from sr.volvox.process import get_orientation_vec
from pathlib import Path
import json
import pandas as pd

import matplotlib.pylab as plt
from sr.plot_util import *

import numpy as np

In [ ]:
results = pd.read_hdf('results2p.h5')
results["R"] = 8.0

results['phase_lag_r'] = results.phase_lack
# m = results.run_type=='final_run_wave_physical2'
sel = results.query('(alpha != 0 or alpha != 180) and (run_type=="final_run_wave_physical2" or run_type=="final_run_wave_physical") ')

results.loc[sel.index, 'phase_lag_r'] = sel.k_theta/np.tan(sel.alpha.astype('float')/180*np.pi)*np.pi/4

#Add a minus sign, to have simplectic waves positive and anti negative
results['phase_lag_r'] = (-results.phase_lag_r + np.pi) % (2 * np.pi) - np.pi

In [ ]:
import pickle
with open('correlation_fits.pickle', 'rb') as filep:
    correlation_fits = pickle.load(filep)
    
with open('correlation_fits_T.pickle', 'rb') as filep:
    correlation_fits_T = pickle.load(filep)


In [ ]:
num_cilia_eq = 7

In [ ]:
sel = results.query(
    f'num_cil_eq=={num_cilia_eq} and run_type=="final_long_smooth"').sort_values('phase_lack')

df = sel.query("k_theta==0").sort_values('v_normal_m').iloc[-1]
phase_lag = df.phase_lag_r
phase_lag/np.pi*180

from scipy.optimize import curve_fit

def f(t, alpha, omega_n, kappa):
    A = np.sin(alpha)
    kappa = np.abs(kappa)
    return (np.cos(omega_n*t)*A**2 + (1-A**2))*np.exp(-kappa*t)

def get_angular_correlation(df):
    data = load_data(Path(df.name))
    T = data[['nx', 'ny', 'nz']]
    T = T.div(np.sqrt((T*T).sum(axis=1)),axis=0)
    q = {}
    for i in range(int(T.last_valid_index()/df.tau)):
        ts = i*df.tau
        q[ts] = ((T*T.shift(int(ts/2.5))).dropna()).sum(axis=1).mean()
    return pd.Series(q)

def fit_angular_correlation(q):
    F = np.fft.rfftfreq(len(q))
    w = F[np.abs(np.fft.rfft(q - q.mean())).argmax()]*2*np.pi
    alpha, omega_n = 0.0, w
    q = q.loc[:q.last_valid_index()/1.5]
    try:
        (alpha, omega_n), pcov = curve_fit(lambda t, a, o: f(t, a, o, 0.0),
                                       q.index,q.values, sigma=1+q.index,
                                           p0=[alpha, omega_n], maxfev=int(1e4))
    except RuntimeError:
        pass
    #(kappa,), pcov = curve_fit(lambda t, k: f(t, alpha, omega_n, k),
    #                                   q.index,q.values, p0=0.0, maxfev=int(1e4))
    (alpha, omega_n, kappa), pcov = curve_fit(f, q.index,q.values,
                                             p0=[alpha, omega_n, 0.0],  sigma=1+q.index,
                                              maxfev=int(1e4))
    
    #,method='trf',                      #bounds=[(0,0,-np.inf),(np.pi, np.inf,np.inf)], 

    alpha = np.abs((alpha + np.pi) % (2*np.pi) - np.pi)
    #kappa = np.abs(kappa)
    return alpha, np.abs(omega_n), kappa



In [ ]:
sel = results.query(
    f'num_cil_eq=={num_cilia_eq} and run_type=="final_long_smooth" and thetaZ==0.0').sort_values('phase_lack')



In [ ]:
sel = results.query(
    f'num_cil_eq=={num_cilia_eq} and run_type=="final_long_smooth" and thetaZ==0.0').sort_values('phase_lack')


correlation_fits = {}
for k_theta, df in sel[sel.phase_lag_r == phase_lag].groupby('k_theta'):
    assert len(df) == 1
    df = df.iloc[0]
    q = get_angular_correlation(df)
    q.index = q.index /df.tau
    a = fit_angular_correlation(q)
    correlation_fits[k_theta] = {'data': q, 'params': a}

In [ ]:
REVT_FULL/2, 2.1

## Figure 5

In [ ]:
fig = plt.figure(figsize=(REVT_FULL, 4.2))
ax = plt.subplot2grid((2,2), (0,0))
bx = plt.subplot2grid((2,2), (0,1))
cx = plt.subplot2grid((2,2), (1,0))
dx = plt.subplot2grid((2,2), (1,1))



ax.text(-0.1, 1.1, 'a)', transform=ax.transAxes)
for k_theta, df in sel.groupby('k_theta'):
    df = df.sort_values('phase_lag_r')
    # df.k_theta/np.tan(float(df.alpha/180*np.pi))*np.pi/4
    ax.plot(df.phase_lag_r, df.v_normal_m/df.R*df.tau, 'o-',
        label=f"$k_\\theta={k_theta}$")
    
    #ax.fill_between(df.phase_lag_r.astype(float),
    #            (df.v_normal_m/df.R*df.tau - df.v_normal_std/df.R*df.tau).astype(float),
    #            (df.v_normal_m/df.R*df.tau + df.v_normal_std/df.R*df.tau).astype(float),
    #               alpha=0.1)
ax.axvline(phase_lag, lw=10, alpha=0.2, c='r')
ax.set_xlabel(r"phase lag $\chi$")
set_angles_x(ax, angles=np.linspace(-180, 180,5), ax_in_rad=True)
ax.set_ylabel(r'$\langle v_n \rangle ~\tau_b/R$')
ax.set_ylim(0.0, 1.0)


bx.text(-0.1, 1.1, 'b)', transform=bx.transAxes)
for k_theta, df in sel.groupby('k_theta'):
    df = df.sort_values('phase_lag_r')
    bx.plot(df.phase_lag_r, df.v_normal_std/df.R*df.tau, 'o-',
        label=f"$k_\\theta={k_theta}$")
bx.axvline(phase_lag, lw=10, alpha=0.2, c='r')
bx.set_xlabel(r"phase lag $\chi$")
set_angles_x(bx, angles=np.linspace(-180, 180,5), ax_in_rad=True)
bx.set_ylabel(r'$\sqrt{\langle(v_n - \langle v_n \rangle )^2 \rangle}  \tau_b/R$')
bx.set_ylim(0.0, 2.0)


cx.text(-0.1, 1.1, 'c)', transform=cx.transAxes)
for k_theta, df in sel.groupby('k_theta'):
    df = df.sort_values('phase_lag_r')
    cx.plot(df.phase_lag_r, df.Omega_n_mean*df.tau, 'o-',
        label=f"$k_\\varphi={k_theta}$")
    cx.fill_between(df.phase_lag_r.astype(float),
                (df.Omega_n_mean*df.tau - df.Omega_n_std*df.tau).astype(float),
                (df.Omega_n_mean*df.tau + df.Omega_n_std*df.tau).astype(float),
                alpha=0.1)
cx.axvline(phase_lag, lw=10, alpha=0.2, c='r')
cx.set_xlabel(r"phase lag $\chi$")
set_angles_x(cx, angles=np.linspace(-180, 180,5), ax_in_rad=True)
set_angles_y(cx, angles=np.linspace(-10, 20,4), ax_in_rad=True)
cx.set_ylabel(r'$\Omega_n \tau_b$')

######## ---------- dx ----------------------------
dx.text(-0.1, 1.1, 'd)', transform=dx.transAxes)
for k_theta, df in sel.groupby('k_theta'):
    df = (df[df.phase_lag_r == phase_lag]).iloc[0]
    q = correlation_fits.get(df.name)
    #q = value #['data']; a = value['params']
    p, = dx.plot(q.index, q,'-', fillstyle='none',
                 label=f'$k_\\varphi = {k_theta}$')
    #dx.plot(q.index, f(q.index, *a),'--', lw=2, c=p.get_color()) #2.97
#dx.set_legend(loc=3, ncol=2)
dx.set_xlim(0,240)
dx.set_ylim(0.35,1.1)
dx.set_xlabel(r'$\tau/\tau_b$')
dx.set_ylabel(r'$<\mathbf{n}(t)\mathbf{n}(t+\tau)>$')
dx.legend(ncol=2, loc=3, fontsize=8)





plt.subplots_adjust(wspace=0.4, hspace=.5)
plt.savefig('figs/physical_straight_cilia_neq7_smooth.pdf')




In [ ]:
df

In [ ]:
df.phase_lag_r == phase_lag

## Figure 6

In [ ]:
sel = results.query(
    f'num_cil_eq=={num_cilia_eq} and run_type=="final_long_smooth" and k_theta==0.0').sort_values('phase_lack')



In [ ]:
fig = plt.figure(figsize=(REVT_FULL, 4.2))
ax = plt.subplot2grid((2,2), (0,0))
bx = plt.subplot2grid((2,2), (0,1))
cx = plt.subplot2grid((2,2), (1,0))
dx = plt.subplot2grid((2,2), (1,1))



ax.text(-0.1, 1.1, 'a)', transform=ax.transAxes)
for thetaZ, df in sel.groupby('thetaZ'):
    df = df.sort_values('phase_lag_r')
    # df.k_theta/np.tan(float(df.alpha/180*np.pi))*np.pi/4
    ax.plot(df.phase_lag_r, df.v_normal_m/df.R*df.tau, 'o-',
        label=f"$\\theta_z={thetaZ*180/np.pi:.0f}$")
    
    #ax.fill_between(df.phase_lag_r.astype(float),
    #            (df.v_normal_m/df.R*df.tau - df.v_normal_std/df.R*df.tau).astype(float),
    #            (df.v_normal_m/df.R*df.tau + df.v_normal_std/df.R*df.tau).astype(float),
    #               alpha=0.1)
ax.axvline(phase_lag, lw=10, alpha=0.2, c='r')
ax.set_xlabel(r"phase lag $\chi$")
set_angles_x(ax, angles=np.linspace(-180, 180,5), ax_in_rad=True)
ax.set_ylabel(r'$\langle v_n \rangle ~\tau_b/R$')
ax.set_ylim(0.0, 1.0)


bx.text(-0.1, 1.1, 'b)', transform=bx.transAxes)
for thetaZ, df in sel.groupby('thetaZ'):
    df = df.sort_values('phase_lag_r')
    bx.plot(df.phase_lag_r, df.v_normal_std/df.R*df.tau, 'o-',
        label=f"$\\theta_z={thetaZ*180/np.pi:.0f}$")
bx.axvline(phase_lag, lw=10, alpha=0.2, c='r')
bx.set_xlabel(r"phase lag $\chi$")
set_angles_x(bx, angles=np.linspace(-180, 180,5), ax_in_rad=True)
bx.set_ylabel(r'$\sqrt{\langle(v_n - \langle v_n \rangle )^2 \rangle}  \tau_b/R$')
bx.set_ylim(0.0, 2.0)


cx.text(-0.1, 1.1, 'c)', transform=cx.transAxes)
for thetaZ, df in sel.groupby('thetaZ'):
    df = df.sort_values('phase_lag_r')
    cx.plot(df.phase_lag_r, df.Omega_n_mean*df.tau, 'o-',
        label=f"$\\theta_z={thetaZ*180/np.pi:.0f}$")
    cx.fill_between(df.phase_lag_r.astype(float),
                (df.Omega_n_mean*df.tau - df.Omega_n_std*df.tau).astype(float),
                (df.Omega_n_mean*df.tau + df.Omega_n_std*df.tau).astype(float),
                alpha=0.1)
cx.axvline(phase_lag, lw=10, alpha=0.2, c='r')
cx.set_xlabel(r"phase lag $\chi$")
set_angles_x(cx, angles=np.linspace(-180, 180,5), ax_in_rad=True)
set_angles_y(cx, angles=np.linspace(-24, 0,4), ax_in_rad=True)
cx.set_ylabel(r'$\Omega_n \tau_b$')

######## ---------- dx ----------------------------
dx.text(-0.1, 1.1, 'd)', transform=dx.transAxes)

for thetaZ, df in sel.groupby('thetaZ'):
    df = (df[df.phase_lag_r == phase_lag]).iloc[0]
    q = correlation_fits.get(df.name)
    #q = value #['data']; a = value['params']
    p, = dx.plot(q.index, q,'-', fillstyle='none',
                 label=f"$\\theta_r={thetaZ*180/np.pi:.0f}$")
                 
    #dx.plot(q.index, f(q.index, *a),'--', lw=2, c=p.get_color()) #2.97
#dx.set_legend(loc=3, ncol=2)
dx.set_xlim(0,240)
dx.set_ylim(0.33,1.1)
dx.set_xlabel(r'$\tau/\tau_b$')
dx.set_ylabel(r'$<\mathbf{n}(t)\mathbf{n}(t+\tau)>$')
dx.legend(ncol=2, loc=3, fontsize=6)







plt.subplots_adjust(wspace=0.4, hspace=.5)
plt.savefig('figs/physical_straight_cilia_neq7_smooth_thetaZ.pdf')
